This notebook is designed to calculate the metrics for the entire project, given catalog and reference catalog, calculate the precision, recall, and F1, for events, p, and s.

In [1]:
import pandas as pd

# 1. load dataset

In [2]:
reference_phase=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/tonga_picks_updated_2023_0426.csv")
reference_phase_melted = reference_phase.melt(id_vars=['originid', 'stacode'], 
                    value_vars=['arrival_time_P', 'arrival_time_S'], 
                    var_name='type', 
                    value_name='time')
reference_phase_melted['type'] = reference_phase_melted['type'].str.replace('arrival_time_', '')
reference_phase_melted["type"]=reference_phase_melted["type"].replace({"P":"p", "S":"s"})
reference_phase_melted["time"]=pd.to_datetime(reference_phase_melted["time"])
reference_phase_melted.rename(columns={"stacode":"sta", "originid":"mp_event_index"}, inplace=True)
reference_phase_melted.dropna(inplace=True)
reference_phase=reference_phase_melted
reference_phase.head(),len(reference_phase)

(  mp_event_index   sta type                       time
 0       11_52111   A01    p 2009-12-02 01:14:41.317870
 1       11_52111  A02W    p 2009-12-02 01:14:42.457750
 2       11_52111   A03    p 2009-12-02 01:14:43.547000
 3       11_52111   A05    p 2009-12-02 01:14:47.038000
 4       11_52111  A06W    p 2009-12-02 01:14:48.857230,
 57108)

In [3]:
reference_catalog=pd.read_csv("/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/tonga_catalog_updated_2023_0426.csv")
reference_catalog.drop(columns=["eventid","author","subset","mb","ISCid"],inplace=True)
reference_catalog["time"]=pd.to_datetime(reference_catalog["time"])
reference_catalog.rename(columns={"originid":"mp_event_index"},inplace=True)
reference_catalog.head(),len(reference_catalog)

(   latitude  longitude     depth                       time mp_event_index
 0  -21.4486  -175.0123   59.8167 2009-12-02 01:14:28.278360       11_52111
 1  -21.0021  -176.0597  189.5062 2009-12-02 02:25:13.607840       11_52113
 2  -20.0291  -175.3772  179.2744 2009-12-02 15:41:32.776970       11_52115
 3  -20.8052  -174.5321   66.1092 2009-12-03 09:43:16.236000       11_52119
 4  -19.4698  -174.9927  146.2841 2009-12-04 17:37:18.418720       11_52124,
 1163)

## 1.1 iteration 1

### 1.1.1 PhaseNet-TF

In [4]:
iteration1_phasenettf_phase=pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_associated_assignment.csv",
        skiprows=1,
        names=[
            "id",
            "date",
            "time",
            "amp",
            "type",
            "prob",
            "event_index",
            "gamma_score",
        ],
        sep=r"\s+",
    )

iteration1_phasenettf_phase["sta"]=iteration1_phasenettf_phase["id"].str.split(".").str[1]
iteration1_phasenettf_phase["time"]=pd.to_datetime(iteration1_phasenettf_phase["date"]+" "+iteration1_phasenettf_phase["time"])
iteration1_phasenettf_phase=iteration1_phasenettf_phase.drop(["date","id","amp","prob"],axis=1)
iteration1_phasenettf_phase.head(),len(iteration1_phasenettf_phase)

(                        time type  event_index  gamma_score   sta
 0 2009-11-01 12:29:38.419900    p           -1    -1.000000  MSVF
 1 2009-11-01 16:03:19.149900    p        31625     0.079182  MSVF
 2 2009-11-01 20:23:03.249900    p        12394     0.068525  MSVF
 3 2009-11-01 20:59:34.149900    p           -1    -1.000000  MSVF
 4 2009-11-01 20:23:15.619900    s        12394     0.052664  MSVF,
 749549)

### 1.1.2 Association

In [5]:
iteration1_association_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_associated_catalog.csv",
        usecols=["time", "longitude", "latitude", "z(km)", "event_index"],
        sep=r"\s+",
    )
iteration1_association_catalog["time"]=pd.to_datetime(iteration1_association_catalog["time"])
iteration1_association_catalog.rename(
        columns={
            "z(km)": "depth",
        },
        inplace=True,
    )
iteration1_association_catalog = iteration1_association_catalog[
        iteration1_association_catalog["event_index"].isin(
            iteration1_phasenettf_phase["event_index"]
            .value_counts()[iteration1_phasenettf_phase["event_index"].value_counts() >= 10]
            .index
        )
    ]
iteration1_association_catalog.head(),len(iteration1_association_catalog)

(                      time       depth  event_index   longitude   latitude
 7  2010-03-22 10:19:17.558   37.395109          857 -174.019518 -22.143588
 10 2010-04-15 01:53:59.089  356.973000         1513 -179.353219 -21.176853
 11 2010-04-15 01:53:49.567  475.973000         1514 -179.209502 -20.870826
 15 2010-04-27 22:08:58.514  316.537000           27 -173.970578 -23.510370
 20 2010-09-23 23:30:12.772  629.213524          538 -177.499262 -18.009047,
 14476)

In [6]:
# select iteration1_association_phase from iteration1_phasenettf_phase with event_index in iteration1_association_catalog
iteration1_association_catalog_event_index_set=set(iteration1_association_catalog["event_index"])
iteration1_association_phase=iteration1_phasenettf_phase[iteration1_phasenettf_phase["event_index"].isin(iteration1_association_catalog_event_index_set)]
iteration1_association_phase.head(),len(iteration1_association_phase)

(                         time type  event_index  gamma_score   sta
 21 2009-11-04 20:43:54.029900    p        15715     0.083852  MSVF
 29 2009-11-06 16:32:12.719900    p        21203     0.070487  MSVF
 41 2009-11-08 08:25:51.549900    p        26282     0.104916  MSVF
 44 2009-11-08 16:39:36.649900    p        53972     0.106067  MSVF
 70 2009-11-10 08:05:06.929900    p         6004     0.088009  MSVF,
 358469)

### 1.1.3 Relocation

In [7]:
iteration1_relocation_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_bootstrapped.csv",
        usecols=[0, 1, 2, 3, 4],
    )
iteration1_relocation_catalog.rename(
        columns={
            "id": "event_index",
        },
        inplace=True,
    )
iteration1_relocation_catalog["time"] = pd.to_datetime(iteration1_relocation_catalog["datetime"])
iteration1_relocation_catalog.drop(columns=["datetime"], inplace=True)
iteration1_relocation_catalog.head(),len(iteration1_relocation_catalog)

(   event_index   latitude   longitude       depth  \
 0           55 -21.078356 -177.229740  365.655880   
 1           68 -16.359084 -174.505760  204.380435   
 2           74 -20.494930 -176.895955  478.382117   
 3           88 -17.442211 -177.878019  646.372351   
 4           93 -18.800355 -175.508731   28.436913   
 
                            time  
 0 2010-01-18 06:48:53.679099136  
 1 2010-01-26 01:47:51.560340480  
 2 2010-06-28 04:23:34.643994112  
 3 2010-04-16 04:21:24.130119936  
 4 2010-07-12 04:08:03.839479552  ,
 9427)

In [8]:
# select iteration1_relocation_phase from iteration1_phasenettf_phase with event_index in iteration1_relocation_catalog
iteration1_relocation_catalog_event_index_set=set(iteration1_relocation_catalog["event_index"])
iteration1_relocation_phase=iteration1_phasenettf_phase[iteration1_phasenettf_phase["event_index"].isin(iteration1_relocation_catalog_event_index_set)]
iteration1_relocation_phase.head(),len(iteration1_relocation_phase)

(                         time type  event_index  gamma_score   sta
 21 2009-11-04 20:43:54.029900    p        15715     0.083852  MSVF
 70 2009-11-10 08:05:06.929900    p         6004     0.088009  MSVF
 72 2009-11-10 08:17:57.669900    p        21695     0.113771  MSVF
 76 2009-11-10 08:05:30.519900    s         6004     0.002691  MSVF
 77 2009-11-10 08:18:55.549900    s        21695     0.108274  MSVF,
 280844)

## 1.2 Iteration 2

### 1.2.1 PhaseNet-TF

In [9]:
iteration2_phasenettf_phase=pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/semi-v1.assignment.csv",
        skiprows=1,
        names=[
            "id",
            "date",
            "time",
            "amp",
            "type",
            "prob",
            "event_index",
            "gamma_score",
        ],
        sep=r"\s+",
    )

iteration2_phasenettf_phase["sta"]=iteration2_phasenettf_phase["id"].str.split(".").str[1]
iteration2_phasenettf_phase["time"]=pd.to_datetime(iteration2_phasenettf_phase["date"]+" "+iteration2_phasenettf_phase["time"])
iteration2_phasenettf_phase=iteration2_phasenettf_phase.drop(["date","id","amp","prob"],axis=1)
iteration2_phasenettf_phase.head(),len(iteration2_phasenettf_phase)

(                        time type  event_index  gamma_score   sta
 0 2009-11-01 04:21:03.099900    p        25214     0.002465  MSVF
 1 2009-11-01 08:07:02.399900    p           -1    -1.000000  MSVF
 2 2009-11-01 12:29:38.529900    p           -1    -1.000000  MSVF
 3 2009-11-01 12:57:23.319900    p           -1    -1.000000  MSVF
 4 2009-11-01 16:03:19.029900    p        12931     0.077535  MSVF,
 1293246)

### 1.2.2 Association

In [10]:
iteration2_association_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/semi-v1.catalog.csv",
        usecols=["time", "longitude", "latitude", "z(km)", "event_index"],
        sep=r"\s+",
    )
iteration2_association_catalog["time"]=pd.to_datetime(iteration2_association_catalog["time"])
iteration2_association_catalog.rename(
        columns={
            "z(km)": "depth",
        },
        inplace=True,
    )
iteration2_association_catalog = iteration2_association_catalog[
        iteration2_association_catalog["event_index"].isin(
            iteration2_phasenettf_phase["event_index"]
            .value_counts()[iteration2_phasenettf_phase["event_index"].value_counts() >= 10]
            .index
        )
    ]
iteration2_association_catalog.head(),len(iteration2_association_catalog)

(                      time      depth  event_index   longitude   latitude
 6  2009-12-21 19:58:28.726  10.058378          360 -179.915481 -16.702746
 21 2010-02-28 02:50:06.245  10.015000         2197 -175.167431 -20.698065
 31 2010-10-05 10:43:02.557  10.000000         3722 -178.080696 -18.928388
 35 2010-08-20 01:49:08.128  15.810072         4419 -176.360205 -21.317971
 36 2010-06-11 12:00:32.439  18.076178         4748 -174.126398 -21.754550,
 26845)

In [11]:
iteration2_association_catalog_event_index_set=set(iteration2_association_catalog["event_index"])
iteration2_association_phase=iteration2_phasenettf_phase[iteration2_phasenettf_phase["event_index"].isin(iteration2_association_catalog_event_index_set)]
iteration2_association_phase.head(),len(iteration2_association_phase)

(                          time type  event_index  gamma_score   sta
 62  2009-11-04 20:43:53.969900    p        98879     0.005614  MSVF
 81  2009-11-06 16:32:12.919900    p        79255     0.071732  MSVF
 105 2009-11-08 08:25:51.529900    p        67540     0.098302  MSVF
 112 2009-11-08 16:39:36.679900    p        98419     0.111880  MSVF
 114 2009-11-08 16:54:48.049900    p         1609     0.106416  MSVF,
 636834)

### 1.2.3 Relocation

In [12]:
iteration2_relocation_catalog = pd.read_csv(
        "/Users/ziyixi/Library/CloudStorage/OneDrive-MichiganStateUniversity/Paper/PhaseNetTF_myturn/PhaseNet-TF-Figures/phasenettf/data/catalog/continious_semi.csv",
        usecols=[0, 1, 2, 3, 4],
    )
iteration2_relocation_catalog.rename(
        columns={
            "id": "event_index",
        },
        inplace=True,
    )
iteration2_relocation_catalog["time"] = pd.to_datetime(iteration2_relocation_catalog["datetime"])
iteration2_relocation_catalog.drop(columns=["datetime"], inplace=True)
iteration2_relocation_catalog.head(),len(iteration2_relocation_catalog)

(   event_index   latitude   longitude       depth  \
 0           49 -24.055055 -178.753175  581.541092   
 1           59 -22.996563 -175.655182   85.202101   
 2           64 -23.101836 -178.642513  554.767645   
 3           78 -21.863043 -175.590572  140.706452   
 4           95 -20.365366 -177.123331  446.403303   
 
                            time  
 0 2010-09-05 06:20:43.693259008  
 1 2010-02-03 07:50:12.698582784  
 2 2010-07-21 13:36:11.897348096  
 3 2010-02-07 15:23:13.122398720  
 4 2010-04-16 07:59:40.670819840  ,
 13799)

In [13]:
iteration2_relocation_catalog_event_index_set=set(iteration2_relocation_catalog["event_index"])
iteration2_relocation_phase=iteration2_phasenettf_phase[iteration2_phasenettf_phase["event_index"].isin(iteration2_relocation_catalog_event_index_set)]
iteration2_relocation_phase.head(),len(iteration2_relocation_phase)

(                          time type  event_index  gamma_score   sta
 62  2009-11-04 20:43:53.969900    p        98879     0.005614  MSVF
 81  2009-11-06 16:32:12.919900    p        79255     0.071732  MSVF
 112 2009-11-08 16:39:36.679900    p        98419     0.111880  MSVF
 163 2009-11-10 08:05:07.049900    p        71320     0.095103  MSVF
 165 2009-11-10 08:17:57.599900    p        14608     0.114541  MSVF,
 422840)

In [14]:
print(len(iteration2_relocation_phase[iteration2_relocation_phase["type"]=="p"]),len(iteration2_relocation_phase[iteration2_relocation_phase["type"]=="s"]))

343247 79593


# Analysis

In [14]:
def calculate_event_based_metrics(df_new_raw:pd.DataFrame,df_ref_raw:pd.DataFrame):
    # calculate precision, recall, and f1-score, example df_new: iteration1_association_catalog, example df_ref: reference_catalog
    df_new=df_new_raw.copy()
    df_ref=df_ref_raw.copy()
    df_new.sort_values(by=["time"],inplace=True)
    df_ref.sort_values(by=["time"],inplace=True)
    
    merged=pd.merge_asof(df_new,df_ref,on="time",direction="nearest",tolerance=pd.Timedelta("30s"))
    merged.dropna(inplace=True)
    merged.drop_duplicates(subset=["mp_event_index"],inplace=True)
    # calculate precision
    tp=len(merged)
    fp=len(df_new)-len(merged)
    fn=len(df_ref)-len(merged)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*precision*recall/(precision+recall)
    return {
        "tp":tp,
        "fp":fp,
        "fn":fn,
        "precision":precision,
        "recall":recall,
        "f1_score":f1_score,
    }

In [15]:
calculate_event_based_metrics(iteration1_association_catalog,reference_catalog)

{'tp': 1152,
 'fp': 13324,
 'fn': 11,
 'precision': 0.0795799944736115,
 'recall': 0.9905417024935511,
 'f1_score': 0.14732399769806256}

In [16]:
calculate_event_based_metrics(iteration1_relocation_catalog,reference_catalog)

{'tp': 1138,
 'fp': 8289,
 'fn': 25,
 'precision': 0.12071708921183834,
 'recall': 0.9785038693035254,
 'f1_score': 0.21491973559962227}

In [17]:
calculate_event_based_metrics(iteration2_association_catalog,reference_catalog)

{'tp': 1156,
 'fp': 25689,
 'fn': 7,
 'precision': 0.043062022723039675,
 'recall': 0.9939810834049871,
 'f1_score': 0.08254784347329336}

In [18]:
calculate_event_based_metrics(iteration2_relocation_catalog,reference_catalog)

{'tp': 1142,
 'fp': 12657,
 'fn': 21,
 'precision': 0.08275962026233785,
 'recall': 0.9819432502149613,
 'f1_score': 0.15265338858441385}

In [19]:
def calculate_phase_based_metrics(df_new_raw:pd.DataFrame,df_ref_raw:pd.DataFrame,phase:str):
    # calculate precision, recall, and f1-score, example df_new: iteration1_association_phase, example df_ref: reference_phase
    df_new=df_new_raw[df_new_raw["type"]==phase].copy()
    df_ref=df_ref_raw[df_ref_raw["type"]==phase].copy()
    df_new.sort_values(by=["time","sta"],inplace=True)
    df_ref.sort_values(by=["time","sta"],inplace=True)
    
    merged=pd.merge_asof(df_new,df_ref,on="time",by="sta",direction="nearest",tolerance=pd.Timedelta("1s"))
    merged.dropna(inplace=True)
    merged.drop_duplicates(subset=["mp_event_index","sta"],inplace=True)
    # calculate precision
    tp=len(merged)
    fp=len(df_new)-len(merged)
    fn=len(df_ref)-len(merged)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*precision*recall/(precision+recall)
    return {
        "tp":tp,
        "fp":fp,
        "fn":fn,
        "precision":precision,
        "recall":recall,
        "f1_score":f1_score,
    }

In [20]:
calculate_phase_based_metrics(iteration1_phasenettf_phase,reference_phase,"p")

{'tp': 40963,
 'fp': 491469,
 'fn': 1293,
 'precision': 0.07693564624214923,
 'recall': 0.969400795153351,
 'f1_score': 0.14255735285929058}

In [21]:
calculate_phase_based_metrics(iteration1_phasenettf_phase,reference_phase,"s")

{'tp': 13907,
 'fp': 203210,
 'fn': 945,
 'precision': 0.06405302210328993,
 'recall': 0.9363722057635335,
 'f1_score': 0.11990395268333268}

In [22]:
calculate_phase_based_metrics(iteration1_association_phase,reference_phase,"p")

{'tp': 40253,
 'fp': 234628,
 'fn': 2003,
 'precision': 0.1464379131333195,
 'recall': 0.9525984475577433,
 'f1_score': 0.2538524360134579}

In [23]:
calculate_phase_based_metrics(iteration1_association_phase,reference_phase,"s")

{'tp': 12580,
 'fp': 71008,
 'fn': 2272,
 'precision': 0.15050007178063837,
 'recall': 0.8470239698357124,
 'f1_score': 0.25558715969118245}

In [24]:
calculate_phase_based_metrics(iteration1_relocation_phase,reference_phase,"p")

{'tp': 39722,
 'fp': 177532,
 'fn': 2534,
 'precision': 0.18283667964686495,
 'recall': 0.940032184778493,
 'f1_score': 0.30613078494085005}

In [25]:
calculate_phase_based_metrics(iteration1_relocation_phase,reference_phase,"s")

{'tp': 12316,
 'fp': 51274,
 'fn': 2536,
 'precision': 0.1936782512973738,
 'recall': 0.829248586049017,
 'f1_score': 0.3140154509064022}

In [26]:
calculate_phase_based_metrics(iteration2_phasenettf_phase,reference_phase,"p")

{'tp': 40857,
 'fp': 1086634,
 'fn': 1399,
 'precision': 0.03623709634932784,
 'recall': 0.9668922756531617,
 'f1_score': 0.06985613128308941}

In [27]:
calculate_phase_based_metrics(iteration2_phasenettf_phase,reference_phase,"s")

{'tp': 12826,
 'fp': 152929,
 'fn': 2026,
 'precision': 0.07737926457723748,
 'recall': 0.8635873956369513,
 'f1_score': 0.1420321471482279}

In [28]:
calculate_phase_based_metrics(iteration2_association_phase,reference_phase,"p")

{'tp': 40282,
 'fp': 486627,
 'fn': 1974,
 'precision': 0.07644963361794921,
 'recall': 0.9532847406285498,
 'f1_score': 0.1415477058497975}

In [29]:
calculate_phase_based_metrics(iteration2_association_phase,reference_phase,"s")

{'tp': 12111,
 'fp': 97814,
 'fn': 2741,
 'precision': 0.11017511939959063,
 'recall': 0.8154457312146512,
 'f1_score': 0.19412231420854806}

In [30]:
calculate_phase_based_metrics(iteration2_relocation_phase,reference_phase,"p")

{'tp': 39724,
 'fp': 303523,
 'fn': 2532,
 'precision': 0.11573007193070879,
 'recall': 0.9400795153351004,
 'f1_score': 0.20608918737337972}

In [31]:
calculate_phase_based_metrics(iteration2_relocation_phase,reference_phase,"s")

{'tp': 11936,
 'fp': 67657,
 'fn': 2916,
 'precision': 0.14996293643913408,
 'recall': 0.8036628063560464,
 'f1_score': 0.2527608661125523}